In [54]:
import tensorflow as tf
import numpy as np
import os 
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, GRU


In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-07-09 16:12:57.562797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 16:12:57.636427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 16:12:57.637588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


We will now look at the task of generator the next character for a given sequence of characters. This is almost identical to the task of next word prediction, however the vocabulary consists of distinct characters and is therefore much smaller. The dataset we will use is the collected works of Shakespeare.

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 0us/step


In [11]:
# read contents of text file as a single string
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(f"Total number of characters in the text: {len(text)}\n")
print(text[:100])

Total number of characters in the text: 1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [16]:
# generate vocabulary of characters (included both upper and lower case english alphabet and punctuation symbols)
vocab = list(set(text))
print(f"Vocabulary length: {len(vocab)}")
print(vocab)

Voicabulary length: 65
['U', 't', 'j', 'H', 'f', 'u', 'O', 'n', 'G', 'W', 'i', 'b', 'v', '-', 'k', 'D', 'T', 'w', ',', 'P', 'x', 'F', 'o', 'A', ';', 'r', 'd', 'V', '?', ' ', 'h', 'Z', 'm', 'g', 'K', 'X', 'z', 'l', 'J', 'R', 'q', '.', 'C', 'N', '!', 'S', 'I', 'a', 'L', '&', 'p', '\n', 'c', "'", 'B', 'e', ':', 'E', 'Q', '3', 's', 'y', '$', 'M', 'Y']


In [19]:
# now we will use a keras StringLookup layer to tokenize sequences of characacters into a sequence of integer character indices
chars_to_index = tf.keras.layers.StringLookup(vocabulary=list(vocab))

In [30]:
test = ["abcdefg"]

# convert a string into a sequence of chars
test = tf.strings.unicode_split(test, input_encoding='UTF-8')

test_indices = chars_to_index(test)
print(test_indices)

<tf.RaggedTensor [[48, 12, 53, 27, 56, 5, 34]]>


In [33]:
# similarly, we will also create another StringLookup layer that performs the inverse operation of converting a sequence of character indices to characters
index_to_chars = tf.keras.layers.StringLookup(vocabulary=chars_to_index.get_vocabulary(), invert=True)

# also define a function for joining the list of characters into a string
def text_from_indices(idx):
    return tf.strings.reduce_join(index_to_chars(idx), axis=-1)

print(text_from_indices(test_indices)) 

tf.Tensor([b'abcdefg'], shape=(1,), dtype=string)


For our next character prediction task, we will use a many-to-many RNN model so that the input and output sequences will be of the same length. To generate the input and output sequences, we can take a sequence of charcaters of length `sequence_len+1`, then assign the first `sequence_len` charcters as the input and the last `sequence_len` charcters as the output labels (i.e. the output labels are just the input sequence shifted forward by 1). E.g.

`sequence = ['T', 'e', 'n', 's', 'o', 'r']`

`x = ['T', 'e', 'n', 's', 'o']`

`y = ['e', 'n', 's', 'o', 'r']`

To prepare the dataset, we will therefore split the text into chunks of size `sequence_len+1` and create `(x,y)` input-label pair training examples from these.


In [34]:
# first convert the text into a sequence of character indices
all_indices = chars_to_index(tf.strings.unicode_split(text, input_encoding='UTF-8'))
print(all_indices)

tf.Tensor([22 11 26 ... 34 42 52], shape=(1115394,), dtype=int64)


In [53]:
# dataset hyper parameters
SEQ_LENGTH = 100 # fixed sequence length for training input
BATCH_SIZE = 64  # number of examples per batch

In [45]:
# next, we will use the tensorflow Dataset API to work with a stream of charcter indices and generate chunks 
all_indices_dataset = tf.data.Dataset.from_tensor_slices(all_indices)

# create chunks
sequences = all_indices_dataset.batch(SEQ_LENGTH+1, drop_remainder=True) # set drop remainder flag to ensure all batches are of the same size and any smaller remainder batch is discarded

for seq in sequences.take(2):
    print(index_to_chars(seq))
    print()
    print(text_from_indices(seq).numpy())
    print()

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

tf.Tensor(
[b'a' b'r' b'e' b' ' b'a' b'l' b'l' b' ' b'r' b'e' b's' b'o' b'l' b'v'
 b'e' b'd' b' ' b'r' b'a' b't' b'h' b'e' b'r' b' ' b't' b'o' b' ' b'd'
 b'i' b'e' b' ' b't' b'h' b'a' b'n' b' ' b't' b'o' b' ' b'f' b'a' b'm'
 b'i' b's' b'h' b'?' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'R' b'e' b's'
 b'o' b'l' b'v' b'e' b'd' b

2023-07-09 17:13:41.864796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [46]:
# now lets split the seqnuences into input-target pairs
def split_seqeunce(sequence):
    input = sequence[:-1]
    target = sequence[1:]
    return input, target

dataset = sequences.map(split_seqeunce)    

In [48]:
for input, target in dataset.take(2):
    print(f"input: {text_from_indices(input).numpy()}")
    print(f"target: {text_from_indices(target).numpy()}")
    print()

input: b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

input: b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
target: b're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'



2023-07-09 17:21:56.406809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [50]:
# finally, we want to create batches of the chunks and also shuffle the chunks around (the shuffling is done on a fixed smaller buffer size to avoid loading the entire data into memory for the shuffling, also we enalbe prefetching of batches to improve memory access latency during training)
dataset = dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [51]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Now that we've prepared the training dataset, we can build our model. Instead of using a Keras Sequential model, we are going to build our own custom model class for easier access to the states from all the RNN cells which will be needed later for text generation.

In [56]:
VOCAB_SIZE = len(chars_to_index.get_vocabulary())

# model hyperparameters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

In [60]:
class customModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)

        # initilize the layers
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = Dense(vocab_size) # note that we don't include a softmax activation here and instead will set from_logist=True in our loss function

    def call(self, inputs, states=None, return_states=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_states:
            return x, states
        else:
            return x

In [259]:
# model with 2 RNN layers
class customModel2(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn1_units, rnn2_units):
        super().__init__(self)

        # initilize the layers
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru1 = GRU(rnn1_units, return_sequences=True, return_state=True)
        self.gru2 = GRU(rnn2_units, return_sequences=True, return_state=True)
        self.dense1 = Dense(2*vocab_size, activation='relu')
        self.dense2 = Dense(vocab_size) # note that we don't include a softmax activation here and instead will set from_logist=True in our loss function

    def call(self, inputs, states=None, return_states=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states1 = self.gru1.get_initial_state(x)
            states2 = self.gru2.get_initial_state(x)
        else:
            states1, states2 = states
        x, states1 = self.gru1(x, initial_state=states1, training=training)
        x, states2 = self.gru2(x, initial_state=states2, training=training)
        x = self.dense1(x, training=training)
        x = self.dense2(x, training=training)

        if return_states:
            return x, (states1, states2)
        else:
            return x

In [267]:
# build the model
model = customModel2(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, RNN_UNITS)

In [268]:
# now lets test the model by running an input batch through it and inspecting the output shape
for input_example_batch, target_example_batch in dataset.take(1):
    # compute prediction
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
    # since the prediction for each character in the sequence is a probability distribution over the vocabulary, we can sample from the distribution to get a concrete predicted character. We will only do this for the first senquence in the batch
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices, axis=-1)

    # lets compare the prediction to the target for the first sequence in the batch 
    print(f"input: {text_from_indices(input_example_batch[0])}")
    print(f"target: {text_from_indices(target_example_batch[0])}")
    print(f"prediction: {text_from_indices(sampled_indices)}")

2023-07-09 21:33:02.014676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-07-09 21:33:02.014908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


(64, 100, 66) # (batch_size, sequence_length, vocab_size)
input: b" I desire my life\nOnce more to look on him.\n\nFLORIZEL:\nBy his command\nHave I here touch'd Sicilia an"
target: b"I desire my life\nOnce more to look on him.\n\nFLORIZEL:\nBy his command\nHave I here touch'd Sicilia and"
prediction: b"hk3nF\nTog[UNK]Pm\nyRwDGWKChXeC?':FIZZGDpDBw?pTBxCHrbT!B!FYUN$AGMryXmWanINOQL:.e?ysqQ$[UNK]xNYdsz;r&W?yEpCPK!r"


Note that the predicted next characters are complete gibberish because the model weights are randomly initialized and haven't been trained yet

In [269]:
model.summary()

Model: "custom_model2_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    multiple                  16896     
                                                                 
 gru_22 (GRU)                multiple                  3938304   
                                                                 
 gru_23 (GRU)                multiple                  6297600   
                                                                 
 dense_15 (Dense)            multiple                  135300    
                                                                 
 dense_16 (Dense)            multiple                  8778      
                                                                 
Total params: 10,396,878
Trainable params: 10,396,878
Non-trainable params: 0
_________________________________________________________________


Note that this model can accomodate input sequences of arbitrary length and so the output shape is shown as 'multiple'

In [270]:
# loss function (set the from_logits flag to True since the output layer computes logits, i.e. its a linear layer)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# test the loss function on the prevoiusly computed example prediction
example_loss = loss(target_example_batch, example_batch_predictions)
print(f"Mean loss: {example_loss}")
print(f"exp(Mean loss): {np.exp(example_loss)} => approximately equal to vocab size")

Mean loss: 4.189546585083008
exp(Mean loss): 65.99285888671875 => approximately equal to vocab size


In [271]:
# compile the model
model.compile(loss=loss, optimizer='adam')

In [272]:
# for larger models, its safe practice to frequently dump checkpoints of trained weights on disk
checkpoint_dir = "./training_checkpoints"
# checkpoint file name
checkpoint_prefix = os.path.join(checkpoint_dir, "chkpt_{epoch}")

# callback function for dumping weights to checkpoint files
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [273]:
NUM_EPOCHS = 50

# train the model
history = model.fit(dataset, epochs=NUM_EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50


2023-07-09 21:33:16.156688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 21:33:16.157943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 21:33:16.158852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

172/172 [==============================] - 11s 49ms/step - loss: 2.4920
Epoch 2/50
172/172 [==============================] - 9s 47ms/step - loss: 1.7226
Epoch 3/50
172/172 [==============================] - 9s 47ms/step - loss: 1.4941
Epoch 4/50
172/172 [==============================] - 9s 47ms/step - loss: 1.3920
Epoch 5/50
172/172 [==============================] - 9s 47ms/step - loss: 1.3300
Epoch 6/50
172/172 [==============================] - 9s 48ms/step - loss: 1.2807
Epoch 7/50
172/172 [==============================] - 9s 47ms/step - loss: 1.2407
Epoch 8/50
172/172 [==============================] - 9s 48ms/step - loss: 1.1992
Epoch 9/50
172/172 [==============================] - 9s 48ms/step - loss: 1.1576
Epoch 10/50
172/172 [==============================] - 9s 48ms/step - loss: 1.1122
Epoch 11/50
172/172 [==============================] - 9s 47ms/step - loss: 1.0628
Epoch 12/50
172/172 [==============================] - 9s 48ms/step - loss: 1.0043
Epoch 13/50
172/172 [==

Using the trained model to generate text: We will now create a one-step model that computes outputs for a single forward pass

In [274]:
class oneStep(tf.keras.Model):
    def __init__(self, model, chars_to_index, index_to_chars, temperature=1.0):
        super().__init__(self)
        self.model = model
        self.chars_to_index = chars_to_index
        self.index_to_chars = index_to_chars
        self.temperature = temperature

        # create a mask to filter out any OOV tokens generated 
        # the mask is just a vector of length equal to vocabulary size with -infinity at the position corresponding to the '[UNK]' 
        # character and zeros everywhere else this vector will be added to the model output predictions so that the logit which 
        # corresponds to the [UNK] token is always -infinity and therefore always has zero probability
        UNK_position = self.chars_to_index(['[UNK]']).numpy()[0]
        prediction_mask = np.zeros(shape=(VOCAB_SIZE))
        prediction_mask[UNK_position] = -float('inf')
        prediction_mask = tf.convert_to_tensor(prediction_mask, dtype='float32')

    # use tensorflow function decorator for performance boost
    @tf.function
    def generate_one_step(self, input_text, states=None):
        # convert the input text to sequence of character indices
        input_chars = tf.strings.unicode_split(input_text, input_encoding='UTF-8')
        input_indices = self.chars_to_index(input_chars).to_tensor()

        # comute prediction
        predicted_logits, states = self.model(inputs=input_indices, states=states, return_states=True)

        # since we only want the predicted next character after the input sequence, we just keep the last character from the predicted sequence
        print(predicted_logits)
        predicted_logits = predicted_logits[:,-1,:]      
        predicted_logits = predicted_logits / self.temperature  # normalise by the temperture parameter

        # apply mask to prevent [UNK] from being generated
        predicted_logits = predicted_logits + prediction_mask

        # now sample the predicted probability distribution 
        predicted_indices = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_indices = tf.squeeze(predicted_indices, axis=-1)

        # convert the predicted indices to characters
        predicted_chars = self.index_to_chars(predicted_indices)

        # return the predicted characters and model state 
        # (the model state will be reused to generate more characters in the sequence)
        return predicted_chars, states


In [278]:
# instantiate a one-step model with default temperature parameter value
one_step_model = oneStep(model, chars_to_index, index_to_chars, temperature = 0.9)

In [279]:
def generate_text(input_text, numChars):

    print(f"Generating {numChars} characters from seed '{input_text}'")
    generated_text = input_text
    input_text = tf.constant([input_text])
    states = None

    start_time = time.time()
    # generate the specified number of characters
    for i in range(numChars):
        input_text, states = one_step_model.generate_one_step(input_text, states)
        next_char = input_text.numpy()[0].decode("utf-8")
        generated_text += next_char
        #print(f"Predicted next character = {next_char}")
        

    end_time = time.time()
    print(f"Text generation run time: {end_time-start_time} seconds")

    return generated_text

In [280]:
# test the text generator
seed_text = "ROMEO:"
numChars = 1000

generated_text = generate_text(seed_text, numChars)
print(f"\nSeed text: '{seed_text}' \nGenerated text: \n\n{generated_text}")

Generating 1000 characters from seed 'ROMEO:'


2023-07-09 22:32:00.380105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 22:32:00.384235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 22:32:00.385883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Tensor("custom_model2_9/dense_16/BiasAdd:0", shape=(1, None, 66), dtype=float32)


2023-07-09 22:32:00.917129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 22:32:00.918961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 22:32:00.920400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Tensor("custom_model2_9/dense_16/BiasAdd:0", shape=(1, None, 66), dtype=float32)
Text generation run time: 9.821748971939087 seconds

Seed text: 'ROMEO:' 
Generated text: 

ROMEO:
Whence art thou and perform'd!

COMINIUS:
You have fought together.

MENENIUS:
One word more, one word.
This till the veriest way they are true.

AUTOLYCUS:
Here's such adversion to despair an enemy's,
Is very nature and the law,
And tutor us to't.

POLIXENES:
Masters, lady! lady!
Ale, hold, here Cominius, sir; here cut thyself
Had been by, to resign executed, and still their powers
Of citizens.

First Citizen:
I think 'twill serve, if any thing that I have,
To seek the panty and the man give thee.

LADY GREY:
To do them good and bad, that cannot do't.

FLORIZEL:
Whence are you?
Here's no place doth lends them sit so.

GLOUCESTER:
I know so. But, gentle Lady:
Of every dost thou make done with a seas
Than a traitor and a little while!
You have wounds come to the sun.
Could we but long till twice a bar and loss